In [2]:
import ast
from dataclasses import dataclass
from typing import List, Tuple, Set, Dict
from collections import defaultdict
import json

In [3]:
import importlib.util
module_path = "dsl.py"
spec = importlib.util.spec_from_file_location("function_module", module_path)
module = importlib.util.module_from_spec(spec)
spec.loader.exec_module(module)


In [4]:
from itertools import combinations
import networkx as nx
import ast
from dataclasses import dataclass
from typing import List, Set, Dict, Tuple
from itertools import combinations
import networkx as nx

@dataclass(frozen=True)
class Operation:
    """Represents an incremental operation in the function composition"""
    input_expr: str
    output_expr: str
    function_str: str
    depth: int

    def __str__(self):
        return f"{self.input_expr} -> {self.output_expr}"

class FunctionDecomposer:
    def __init__(self, module_content: str):
        self.operations: Set[Operation] = set()
        self.variable_map: Dict[str, str] = {}
        self.counter = 0
        self.function_names = self._extract_function_names(module_content)
    
    def _extract_function_names(self, module_content: str) -> Set[str]:
        """Extract all function names defined in the module"""
        tree = ast.parse(module_content)
        return {node.name for node in ast.walk(tree) if isinstance(node, ast.FunctionDef)}
    
    def get_fresh_variable(self) -> str:
        """Generate a fresh variable name"""
        self.counter += 1
        return f"temp_{self.counter}"
    
    def is_function_ref(self, name: str) -> bool:
        """Check if a name refers to a function defined in the module"""
        return name in self.function_names
    
    def extract_operations(self, expr_str: str) -> List[Operation]:
        """Extract all incremental operations from a function expression"""
        tree = ast.parse(expr_str).body[0].value
        self.operations.clear()
        self.variable_map.clear()
        self.counter = 0
        self._process_node(tree, "I", 0)
        return sorted(self.operations, key=lambda x: (x.depth, x.output_expr))
    
    def _process_node(self, node: ast.AST, input_var: str, depth: int) -> Tuple[str, bool]:
        """
        Process an AST node and extract operations.
        Returns (expression_string, is_function_reference)
        """
        if isinstance(node, ast.Name):
            is_func = self.is_function_ref(node.id)
            return node.id, is_func
        
        if isinstance(node, ast.Call):
            func_name, _ = self._process_node(node.func, input_var, depth)
            
            # Process arguments
            processed_args = []
            for arg in node.args:
                arg_expr, is_func_ref = self._process_node(arg, input_var, depth + 1)
                
                # Only create variables for non-function, non-constant values
                if not (is_func_ref or arg_expr.isupper() or arg_expr == "I"):
                    var_name = self.get_fresh_variable()
                    self.variable_map[arg_expr] = var_name
                    # Create operation for the intermediate result
                    self.operations.add(Operation(
                        input_expr="I",
                        output_expr=arg_expr,
                        function_str=f"lambda x: {arg_expr}",
                        depth=depth + 1
                    ))
                    processed_args.append(var_name)
                else:
                    processed_args.append(arg_expr)
            
            # Create the function call string
            func_call = f"{func_name}({', '.join(processed_args)})"
            
            # Create operation for this function call
            self.operations.add(Operation(
                input_expr="I",
                output_expr=func_call,
                function_str=f"lambda x: {func_call}",
                depth=depth
            ))
            
            return func_call, False  # Function call is not a function reference
        
        return input_var, False

    def _remove_unnecessary_parentheses(self, expr: str) -> str:
        """Remove unnecessary parentheses from expression"""
        stack = []
        indices_to_remove = set()
        last_open = -1
        
        # Find matching parentheses pairs
        for i, c in enumerate(expr):
            if c == '(':
                stack.append(i)
            elif c == ')' and stack:
                start = stack.pop()
                # Check if these parentheses are unnecessary
                inner = expr[start+1:i]
                if not inner.strip():  # Empty
                    indices_to_remove.add(start)
                    indices_to_remove.add(i)
                elif inner.count('(') == inner.count(')'):  # Balanced inner expression
                    if not any(c in inner for c in ', '): # No commas inside
                        # Check if inner contains only a simple expression
                        if not any(op in inner for op in ['+', '-', '*', '/', ' ']):
                            indices_to_remove.add(start)
                            indices_to_remove.add(i)
        
        # Rebuild string without unnecessary parentheses
        return ''.join(c for i, c in enumerate(expr) if i not in indices_to_remove)

    def _substitute_variables(self, expr: str, substitutions: Dict[str, str], seen: Set[str] = None) -> str:
        """
        Substitute variables in expression while avoiding circular dependencies.
        Returns cleaned-up expression with all possible substitutions made.
        """
        if seen is None:
            seen = set()
        
        # Find all variables that need to be substituted
        changes_made = True
        while changes_made:
            changes_made = False
            for var, sub_expr in substitutions.items():
                if var in expr and var not in seen:
                    seen.add(var)
                    # Recursively substitute in the replacement expression first
                    sub_expr = self._substitute_variables(sub_expr, substitutions, seen)
                    # Only add parentheses if the substituted expression contains operators
                    needs_parens = any(op in sub_expr for op in ['+', '-', '*', '/', ' ']) and not (sub_expr.startswith('(') and sub_expr.endswith(')'))
                    wrapped_expr = f"({sub_expr})" if needs_parens else sub_expr
                    expr = expr.replace(var, wrapped_expr)
                    changes_made = True
                    
        return self._remove_unnecessary_parentheses(expr)

    def get_all_granularities(self) -> List[List[Operation]]:
        """
        Generate all possible granularities of the solution by merging operations.
        Returns a list of lists, where each inner list represents one possible granularity level
        ordered from finest (original operations) to coarsest (fully merged).
        """
        # Build dependency graph
        G = nx.DiGraph()
        var_to_op = {}
        
        # Add all operations as nodes
        for op in self.operations:
            G.add_node(op)
            if op.output_expr in self.variable_map:
                var_to_op[self.variable_map[op.output_expr]] = op
        
        # Add edges based on variable dependencies
        for op in self.operations:
            for var_name in self.variable_map.values():
                if var_name in op.function_str:
                    if var_name in var_to_op:
                        G.add_edge(var_to_op[var_name], op)
        
        def merge_operations(ops_to_merge: Set[Operation]) -> Operation:
            """Merge a set of operations into a single operation"""
            sorted_ops = list(nx.topological_sort(G.subgraph(ops_to_merge)))
            
            # Build complete substitutions map
            substitutions = {}
            for op in sorted_ops:
                if op.output_expr in self.variable_map:
                    var_name = self.variable_map[op.output_expr]
                    substitutions[var_name] = op.output_expr
            
            # Get the final expression and substitute all variables
            final_expr = sorted_ops[-1].output_expr
            merged_expr = self._substitute_variables(final_expr, substitutions)
            
            return Operation(
                input_expr="I",
                output_expr=merged_expr,
                function_str=f"lambda x: {merged_expr}",
                depth=min(op.depth for op in ops_to_merge)
            )
        
        def merge_operations(ops_to_merge: Set[Operation]) -> Operation:
            """Merge a set of operations into a single operation"""
            # Sort operations by dependency order
            sorted_ops = list(nx.topological_sort(G.subgraph(ops_to_merge)))
            
            # Create merged function string by substituting variables
            final_expr = sorted_ops[-1].output_expr
            substitutions = {}
            
            # Build substitutions map
            for op in sorted_ops[:-1]:
                if op.output_expr in self.variable_map:
                    var_name = self.variable_map[op.output_expr]
                    substitutions[var_name] = op.output_expr
            
            # Apply substitutions from deepest to shallowest
            merged_expr = final_expr
            while any(var in merged_expr for var in substitutions):
                for var, expr in substitutions.items():
                    merged_expr = merged_expr.replace(var, f"({expr})")
            
            return Operation(
                input_expr="I",
                output_expr=merged_expr,
                function_str=f"lambda x: {merged_expr}",
                depth=min(op.depth for op in ops_to_merge)
            )
        
        def is_valid_merge(ops_to_merge: Set[Operation]) -> bool:
            """Check if a set of operations can be validly merged"""
            # Check if subgraph is connected
            subgraph = G.subgraph(ops_to_merge)
            return nx.is_weakly_connected(subgraph)
        
        def get_valid_merges(n: int) -> List[Set[Operation]]:
            """Get all valid combinations of n operations that can be merged"""
            valid_merges = []
            for ops in combinations(self.operations, n):
                ops_set = set(ops)
                if is_valid_merge(ops_set):
                    valid_merges.append(ops_set)
            return valid_merges
        
        # Generate all possible granularities
        all_granularities = []
        
        # Start with finest granularity (original operations)
        finest = sorted(self.operations, key=lambda x: (x.depth, x.output_expr))
        all_granularities.append(finest)
        
        # Generate coarser granularities by merging operations
        for size in range(2, len(self.operations) + 1):
            valid_merges = get_valid_merges(size)
            if not valid_merges:
                continue
                
            granularity = []
            used_ops = set()
            
            # For each valid merge
            for merge_group in valid_merges:
                # Skip if any operation in this group has been used
                if any(op in used_ops for op in merge_group):
                    continue
                    
                # Merge the operations
                merged_op = merge_operations(merge_group)
                granularity.append(merged_op)
                used_ops.update(merge_group)
                
            # Add remaining operations that weren't merged
            remaining_ops = [op for op in self.operations if op not in used_ops]
            granularity.extend(remaining_ops)
            
            # Sort operations at this granularity level
            granularity.sort(key=lambda x: (x.depth, x.output_expr))
            
            # Add this granularity level if it's different from previous ones
            if granularity and granularity not in all_granularities:
                all_granularities.append(granularity)
        
        return all_granularities

def generate_granular_problems(expr_str: str, module_content: str) -> Dict[str, Dict[str, str]]:
    """
    Generate problems at all possible granularities.
    Returns a dictionary mapping granularity level to a dictionary of problems.
    """
    decomposer = FunctionDecomposer(module_content)
    decomposer.extract_operations(expr_str)
    
    all_granularities = decomposer.get_all_granularities()
    
    problems = {}
    for i, granularity in enumerate(all_granularities):
        level_problems = {}
        for j, op in enumerate(granularity):
            problem_id = f"gran_{i}_step_{j:03d}"
            level_problems[problem_id] = op.function_str
        problems[f"level_{i}"] = level_problems
    
    return problems

def generate_incremental_problems(expr_str: str, module_content: str) -> Dict[str, str]:
    """Generate incremental problems from a function expression."""
    decomposer = FunctionDecomposer(module_content)
    operations = decomposer.extract_operations(expr_str)
    
    problems = {}
    for i, op in enumerate(operations):
        problem_id = f"inc_{i:03d}"
        problems[problem_id] = op.function_str
    
    return problems


In [5]:
from dataclasses import dataclass
from typing import List, Dict, Set
import ast

@dataclass
class Step:
    expression: str
    result_var: str
    dependencies: Set[str] = None

    def __post_init__(self):
        if self.dependencies is None:
            self.dependencies = set()

class Solution:
    def __init__(self):
        self.steps: List[Step] = []
        self.temp_counter = 0
        self.var_map: Dict[str, str] = {}
        
    def add_step(self, expr: str, dependencies: Set[str] = None) -> str:
        if expr in self.var_map:
            return self.var_map[expr]
        result_var = f"temp_{self.temp_counter}"
        self.temp_counter += 1
        step = Step(expr, result_var, dependencies or set())
        self.steps.append(step)
        self.var_map[expr] = result_var
        return result_var

    def __str__(self) -> str:
        return "\n".join(f"step {i}: {step.result_var} = {step.expression}" 
                        for i, step in enumerate(self.steps))

    def reorder_steps(self):
        """Reorder steps based on dependencies"""
        dependency_graph = {step.result_var: step.dependencies for step in self.steps}
        visited = set()
        ordered_steps = []

        def visit(var):
            if var in visited:
                return
            visited.add(var)
            for dep in dependency_graph.get(var, []):
                visit(dep)
            step = next(s for s in self.steps if s.result_var == var)
            ordered_steps.append(step)

        for step in self.steps:
            visit(step.result_var)

        self.steps = ordered_steps

In [6]:
import ast
from dataclasses import dataclass
from typing import List, Dict, Set

def get_expression_str(node: ast.AST) -> str:
    """Convert an AST node back to its string representation."""
    if isinstance(node, ast.Name):
        return node.id
    elif isinstance(node, ast.Call):
        args = [get_expression_str(arg) for arg in node.args]
        return f"{node.func.id}({', '.join(args)})"
    else:
        raise ValueError(f"Unexpected node type: {type(node)}")

def collect_calls(node: ast.AST) -> List[ast.Call]:
    """Collect all Call nodes in order from innermost to outermost."""
    calls = []
    
    def visit(node):
        if isinstance(node, ast.Call):
            # First visit all arguments
            for arg in node.args:
                visit(arg)
            # Then add this call
            calls.append(node)
        return node
        
    visit(node)
    return calls

def substitute_expressions(expr: str, expr_map: Dict[str, str]) -> str:
    """Repeatedly substitute expressions until no more substitutions can be made."""
    prev_expr = None
    current_expr = expr
    
    while prev_expr != current_expr:
        prev_expr = current_expr
        # Sort expressions by length (longest first) to avoid partial replacements
        for old_expr, var in sorted(expr_map.items(), key=lambda x: len(x[0]), reverse=True):
            if old_expr in current_expr and old_expr != current_expr:
                current_expr = current_expr.replace(old_expr, var)
    
    return current_expr

def generate_solutions(expr: str) -> List[Solution]:
    # Parse the expression
    tree = ast.parse(expr, mode='eval')
    
    # Get all function calls from innermost to outermost
    calls = collect_calls(tree.body)
    
    # Create solutions
    solutions = []
    
    # Solution 0: Just the complete expression
    solution = Solution()
    solution.add_step(get_expression_str(tree.body))
    solutions.append(solution)
    
    # Create increasingly granular solutions
    for i in range(len(calls) - 1, -1, -1):
        solution = Solution()
        expr_map = {}
        
        # Process all calls from index i onwards
        for call in calls[i:]:
            # Convert the call to string form and substitute all known expressions
            current_expr = get_expression_str(call)
            current_expr = substitute_expressions(current_expr, expr_map)
            
            # Add this step and remember the mapping
            temp_var = solution.add_step(current_expr)
            expr_map[get_expression_str(call)] = temp_var
            
            # Update any previous steps that could use this new variable
            for step in solution.steps[:-1]:  # Skip the step we just added
                step.expression = substitute_expressions(step.expression, expr_map)
        
        if len(solution.steps) > 1:  # Only add if we've broken something down
            solutions.append(solution)
    
    return solutions

def print_all_solutions(expr: str):
    solutions = generate_solutions(expr)
    for i, solution in enumerate(solutions):
        print(f"\nSolution {i} ({'Most' if i == len(solutions)-1 else 'Least' if i == 0 else 'More'} Granular):")
        print(solution)

In [7]:
import ast
from typing import Dict, Any, List, Tuple
import inspect
import dsl

def format_collection(collection, indent=0):
    """Format a collection (set, frozenset, tuple) with readable output"""
    if len(collection) == 0:
        return "empty"
        
    items = list(collection)
    try:
        items.sort()
    except TypeError:
        pass
    
    formatted_items = []
    indent_str = " " * (indent + 2)
    
    for item in items:
        
        if isinstance(item, frozenset):
            formatted_items.append(format_collection(item, indent + 2))
        else:
            formatted_items.append(str(item))
            
    items_str = f",\n{indent_str}".join(formatted_items)
    return "{\n" + indent_str + items_str + "\n" + (" " * indent) + "}"

def format_result(result):
    if isinstance(result, tuple):
        if len(result) == 0:
            return "()"
        if isinstance(result[0], tuple):
            return f"Grid {len(result)}x{len(result[0])}"
        if len(result) == 2 and all(isinstance(x, int) for x in result):
            return f"Point/Vector {result}"
        return f"Tuple {format_collection(result)}"
    elif isinstance(result, (set, frozenset)):
        unique_elements = set(result)
        size_str = f"FrozenSet of size {len(unique_elements)}"
        elements_str = format_collection(unique_elements)
        return f"{size_str}\nElements: {elements_str}"
    elif callable(result):
        return f"Function {result.__name__}"
    return str(result)

def print_grid(grid):
    if not isinstance(grid, tuple) or not grid:
        print(grid)
        return
    
    if not isinstance(grid[0], tuple):
        print(grid)
        return
    
    print(f"Grid {len(grid)}x{len(grid[0])}:")
    for row in grid:
        print(" ".join(str(x) for x in row))
    print()

def format_input_value(value):
    """Format an input value for display"""
    if isinstance(value, tuple) and value and isinstance(value[0], tuple):
        return f"Grid {len(value)}x{len(value[0])}"
    return format_result(value)

In [8]:
import ast
from typing import Dict, Any, List, Tuple, Set
import inspect
import dsl
import re

class StepEvaluator:
    def __init__(self):
        self.functions = {
            name: func for name, func in inspect.getmembers(dsl, inspect.isfunction)
            if not name.startswith('_')
        }
        self.constants = {
            'NEG_ONE': -1, 'NEG_TWO': -2, 'ZERO': 0, 'ONE': 1, 'TWO': 2,
            'THREE': 3, 'FOUR': 4, 'FIVE': 5, 'SIX': 6, 'SEVEN': 7,
            'EIGHT': 8, 'NINE': 9, 'TEN': 10, 'DOWN': (1, 0), 'RIGHT': (0, 1),
            'UP': (-1, 0), 'LEFT': (0, -1), 'ORIGIN': (0, 0),
            'UNITY': (1, 1), 'NEG_UNITY': (-1, -1), 'UP_RIGHT': (-1, 1),
            'DOWN_LEFT': (1, -1), 'ZERO_BY_TWO': (0, 2),
            'TWO_BY_ZERO': (2, 0), 'TWO_BY_TWO': (2, 2),
            'THREE_BY_THREE': (3, 3), 'T': True, 'F': False
        }
        self.temp_values: Dict[str, Any] = {}
        self.current_step_inputs: Dict[str, Any] = {}
        self.pending_steps: Dict[str, str] = {}
        
    def evaluate_name(self, node: ast.Name) -> Any:
        if node.id in self.temp_values:
            self.current_step_inputs[node.id] = self.temp_values[node.id]
            return self.temp_values[node.id]
        elif node.id == 'I':
            self.current_step_inputs['I'] = self.input_grid
            return self.input_grid
        elif node.id in self.constants:
            return self.constants[node.id]
        elif node.id in self.functions:
            return self.functions[node.id]
        elif node.id in self.pending_steps:
            # Evaluate pending step first
            pending_expr = self.pending_steps[node.id]
            result = self.evaluate_expression(pending_expr)
            self.temp_values[node.id] = result
            self.current_step_inputs[node.id] = result
            return result
        else:
            raise ValueError(f"Unknown variable or function: {node.id}")
        
    def evaluate_call(self, node: ast.Call) -> Any:
        if isinstance(node.func, ast.Name):
            func_name = node.func.id
            if func_name not in self.functions:
                raise ValueError(f"Unknown function: {func_name}")
            func = self.functions[func_name]
        else:
            raise ValueError(f"Unsupported function reference type: {type(node.func)}")
            
        args = []
        for arg in node.args:
            if isinstance(arg, ast.Name):
                args.append(self.evaluate_name(arg))
            elif isinstance(arg, ast.Call):
                args.append(self.evaluate_call(arg))
            else:
                raise ValueError(f"Unsupported argument type: {type(arg)}")
                
        return func(*args)
    
    def evaluate_expression(self, expr: str) -> Any:
        """Evaluate a single expression"""
        tree = ast.parse(expr, mode='eval')
        return self.evaluate_call(tree.body)
    
    def evaluate_step(self, step_expr: str, result_var: str) -> Tuple[Any, Dict[str, Any]]:
        """Evaluate a step and return both result and inputs used"""
        self.current_step_inputs = {}
        result = self.evaluate_expression(step_expr)
        self.temp_values[result_var] = result
        return result, dict(self.current_step_inputs)

    def evaluate_solution(self, solution, input_grid) -> List[Tuple[str, str, Any, Dict[str, Any]]]:
        """Evaluate all steps in a solution"""
        self.input_grid = input_grid
        self.temp_values.clear()
        self.pending_steps.clear()
        
        # First collect all steps
        for step in solution.steps:
            self.pending_steps[step.result_var] = step.expression
            
        step_results = []
        # Then evaluate them in order
        for step in solution.steps:
            try:
                result, inputs = self.evaluate_step(step.expression, step.result_var)
                step_results.append((step.result_var, step.expression, result, inputs))
            except Exception as e:
                print(f"Error evaluating step {step.result_var} = {step.expression}")
                print(f"Current temp_values: {list(self.temp_values.keys())}")
                raise
            
        return step_results

# [Rest of the code remains the same: format_collection, format_result, print_grid, etc.]

def check_callable(r):
    if isinstance(r, (list, tuple, set, frozenset)):
        return all(check_callable(x) for x in r) and len(r) > 0 #empty set would read true
    else:
        return callable(r)

def compute_used(function_steps, inputs, expr):
    return {
        func_var: func_expr 
        for func_var, (func_expr, _) in function_steps.items() 
        if func_var in inputs or exact_pattern_match(func_var, expr)
    }

def exact_pattern_match(pattern_to_find, text):
    # Escape any special regex characters in the pattern
    escaped_pattern = re.escape(pattern_to_find)
    # Add word boundary at the end
    regex_pattern = fr'{escaped_pattern}\b'
    
    return re.search(regex_pattern, text)

def merge_used(function_steps, var, expr, result, inputs):

    final_expr = expr
    final_var = var
    final_result = result
    final_inputs = inputs
    used_in_step = compute_used(function_steps, inputs, expr)
    # print("orig used", used_in_step)
    # print(used_in_step)
    while used_in_step:
        # print("in while",final_expr)
        # 3. Substitute expressions
        new_expr = final_expr
        new_inputs = dict(final_inputs)
        # print(used_in_step, "before")
        for func_var, func_expr in used_in_step.items():
            # Replace function variable with its expression
            new_expr = new_expr.replace(func_var, f"({func_expr})")
            # Update inputs
            new_inputs.update(function_steps[func_var][1])
            new_inputs.pop(func_var, None)
            # used_functions.add(func_var)
        
        used_in_step = compute_used(function_steps, new_inputs, new_expr)
        
        # print(used_in_step, "after", new_expr, new_inputs)
        final_expr = new_expr
        final_inputs = new_inputs

    return (final_var, final_expr, final_result, final_inputs)
    


def merge_function_steps(step_results):
    """
    Merge steps by repeatedly:
    1. Finding steps that return callables
    2. Finding steps that use those callables
    3. Substituting expressions
    4. Renaming variables
    5. Repeating until no callables remain
    6. Removing any unused function-producing steps
    """
    current_steps = list(step_results)
    # print(current_steps, "curr here")
    function_steps = {
        var: (expr, inputs) 
        for var, expr, result, inputs in current_steps 
        # if (callable(result) or (isinstance(result, (list, tuple, set, frozenset)) and all(callable(r) for r in result)))
        if check_callable(result)
    }

    while True:
        # 1. Find steps that return callables

        function_steps = {
        var: (expr, inputs) 
        for var, expr, result, inputs in current_steps 
        # if (callable(result) or (isinstance(result, (list, tuple, set, frozenset)) and all(callable(r) for r in result)))
        if check_callable(result)
        }

        if not function_steps:
            break
                
        # 2. Find steps that use those callables and perform substitution
        new_steps = []
        used_functions = set()
        
        for var, expr, result, inputs in current_steps:
            # print("iter through current")
            # Skip function steps that will be merged
            if var in function_steps:
                continue
            new_steps.append(merge_used(function_steps, var, expr, result, inputs))
        
        # If no changes were made, break
        if len(new_steps) == len(current_steps):
            break
            
        current_steps = new_steps
    # Final cleanup: remove any remaining function-producing steps
    final_steps = []
    function_vars = {var for var, _, result, _ in current_steps if check_callable(result)}
    used_vars = set()
    
    # Collect all variables that are still being used
    for _, expr, _, inputs in current_steps:
        for var in function_vars:
            if var in expr or var in str(inputs):
                used_vars.add(var)
    
    # Keep only non-function steps and function steps that are still used
    final_steps = [
        step for step in current_steps
        if not check_callable(step[2]) or step[0] in used_vars
    ]
    
    return final_steps

def evaluate_all_solutions(solutions, input_grid):
    """Evaluate all solutions with complete cleanup"""
    evaluator = StepEvaluator()
    all_results = []
    
    for i, solution in enumerate(solutions):
        print(f"\nSolution {i} ({'Most' if i == len(solutions)-1 else 'Least' if i == 0 else 'More'} Granular):")
        print(solution)
        print("\nIntermediate Results:")
        
        try:
            # Get original step results
            step_results = evaluator.evaluate_solution(solution, input_grid)
            
            # Merge steps recursively and cleanup
            merged_steps = merge_function_steps(step_results)
            
            # Print merged results
            for var, expr, result, inputs in merged_steps:
                if not check_callable(result):  # Only show non-function results
                    print(f"\n{var} = {expr}")
                    if inputs:
                        print("Inputs:")
                        for input_var, input_val in inputs.items():
                            print(f"  {input_var}: {format_input_value(input_val)}")
                    print(f"Result type: {type(result).__name__}")
                    print("Value:", end=" ")
                    if isinstance(result, tuple) and result and isinstance(result[0], tuple):
                        print_grid(result)
                    else:
                        print("\n" + format_result(result))
                    
            all_results.append(merged_steps)
        except Exception as e:
            print(f"Error in solution {i}:")
            print(str(e))
            print("Continuing with next solution...")
            continue
    
    return all_results

In [9]:
import ast
from typing import Dict

def map_function_returns(file_path: str) -> Dict[str, str]:
    """
    Parse a Python file and map each function name to its return expression.
    
    Args:
        file_path: Path to the Python file to analyze
        
    Returns:
        Dictionary mapping function names to their return expressions as strings
    """
    with open(file_path, 'r') as file:
        source = file.read()
    
    tree = ast.parse(source)
    return_map = {}
    
    class ReturnVisitor(ast.NodeVisitor):
        def visit_FunctionDef(self, node):
            returns = []
            
            # Inner class to find return statements
            class ReturnFinder(ast.NodeVisitor):
                def visit_Return(self, return_node):
                    if return_node.value:  # Ignore bare 'return' statements
                        returns.append(ast.unparse(return_node.value))
            
            # Find all return statements in the function
            ReturnFinder().visit(node)
            
            # Join multiple returns with ' | ' if they exist
            if returns:
                return_map[node.name] = ' | '.join(returns)
            else:
                return_map[node.name] = 'None'  # Functions without explicit returns
    
    ReturnVisitor().visit(tree)
    return return_map

def print_return_map(file_path: str) -> None:
    """
    Print the function return mappings in a readable format.
    
    Args:
        file_path: Path to the Python file to analyze
    """
    return_map = map_function_returns(file_path)
    max_name_length = max(len(name) for name in return_map.keys())
    
    print("\nFunction Return Expressions:")
    print("-" * (max_name_length + 30))
    for func_name, return_expr in sorted(return_map.items()):
        print(f"{func_name:<{max_name_length}} -> {return_expr}")

In [10]:
import os
def get_data(train=True):
    path = f'arc_original/{"training" if train else "evaluation"}'
    data = {}
    for fn in os.listdir(path):
        with open(f'{path}/{fn}') as f:
            data[fn.rstrip('.json')] = json.load(f)
    ast = lambda g: tuple(tuple(r) for r in g)
    return {
        'train': {k: [{
            'input': ast(e['input']),
            'output': ast(e['output']),
        } for e in v['train']] for k, v in data.items()},
        'test': {k: [{
            'input': ast(e['input']),
            'output': ast(e['output']),
        } for e in v['test']] for k, v in data.items()}
    }


In [11]:
from multiprocessing import Pool, cpu_count
from tqdm import tqdm
from functools import partial

def process_function(func_name, map_dict):
    """Process a single function and generate solutions."""
    exp = map_dict[func_name]
    return func_name, generate_solutions(exp)
# takes each function function to subcompoenent. =============================================================
def parallel_generate_solutions(map_dict, num_processes=None):
    """
    Generate solutions in parallel using multiple processes.
    
    Args:
        map_dict: Dictionary mapping function names to expressions
        num_processes: Number of processes to use (defaults to CPU count)
    
    Returns:
        Dictionary mapping function names to their solutions
    """
    if num_processes is None:
        num_processes = cpu_count()

    # Create a partial function with the map_dict argument fixed
    process_func = partial(process_function, map_dict=map_dict)
    
    # Create a process pool
    with Pool(processes=num_processes) as pool:
        # Use imap_unordered for better performance when order doesn't matter
        # Wrap with tqdm for progress tracking
        results = list(tqdm(
            pool.imap_unordered(process_func, map_dict.keys()),
            total=len(map_dict),
            desc="Generating solutions"
        ))
    
    # Convert results list back to dictionary
    return dict(results)

def single_generate_solutions(map_dict):
    results = {}
    
    # Process each function sequentially with progress bar
    for func_name in tqdm(map_dict.keys(), desc="Generating solutions"):
        func_result = process_function(func_name, map_dict)
        results[func_result[0]] = func_result[1]
    
    return results

In [12]:
def process_collection(collection):
    """Format a collection (set, frozenset, tuple) with readable output"""
    items = list(collection)
    try:
        items.sort()
    except TypeError:
        pass
    
    formatted_items = []
    
    for item in items:
        
        if isinstance(item, frozenset):
            formatted_items.append(format_collection(item))
        else:
            formatted_items.append(str(item))
            
    items_str = " ".join(formatted_items)
    return items_str

def process_inputs(inputs):
    return [process_result(inputs[k]) for k in inputs]

def process_outputs(outputs):
    return [process_result(o) for o in outputs]
    
def convert_frozenset_to_set(obj):
    """
    Recursively converts FrozenSets to Sets in a nested data structure.
    First converts everything to lists, then rebuilds with sets.
    
    Args:
        obj: Any Python object that might contain FrozenSets
        
    Returns:
        The same structure with all FrozenSets converted to Sets
    """
    # First pass: convert everything to lists
    def to_lists(obj):
        if isinstance(obj, (str, int, float, bool, type(None))):
            return obj
        elif isinstance(obj, (frozenset, set)):
            return [to_lists(x) for x in obj]
        elif isinstance(obj, (list, tuple)):
            return [to_lists(x) for x in obj]
        elif isinstance(obj, dict):
            return {to_lists(k): to_lists(v) for k, v in obj.items()}
        else:
            return obj

    # Second pass: convert lists back to sets where needed
    def to_sets(obj):
        if isinstance(obj, (str, int, float, bool, type(None))):
            return obj
        elif isinstance(obj, list):
            # Convert all elements first
            converted = [to_sets(x) for x in obj]
            # If this was originally a set/frozenset, make it a set
            if isinstance(obj, (set, frozenset)):
                return set(converted)
            return converted
        elif isinstance(obj, dict):
            return {to_sets(k): to_sets(v) for k, v in obj.items()}
        else:
            return obj

    # Run both passes
    return to_sets(to_lists(obj))
def process_result(result):
    if isinstance(result, (set, frozenset)):
        return convert_frozenset_to_set(result)
    elif callable(result):
        return "function"
    else:
        return result
        

In [13]:
# second step ============================================================================================
def process_solution(solution, name):
    evaluator = StepEvaluator()
    key = name.replace("solve_", "")
    orig_inputs = [ex['input'] for ex in data['train'][key]]

    steps_all_inputs = []
    for I in orig_inputs:
        step_results = evaluator.evaluate_solution(solution, I)
        
        merged_steps = merge_function_steps(step_results)
        # print(merged_steps)
        for step in merged_steps:
            if callable(step[2]):
                print(name)
                print(solution)
                print(step)
        steps_all_inputs.append(merged_steps)
    subroutines = {}
    # return steps_all_inputs
    # print(len(steps_all_inputs[0][0]))
    for i in range(len(steps_all_inputs[0])):
        inputs = [process_inputs(merged[i][3]) for merged in steps_all_inputs]
        outputs = [process_result(merged[i][2]) for merged in steps_all_inputs]
        program = f"{steps_all_inputs[0][i][0]} = {steps_all_inputs[0][i][1]}"
        subroutines[f"subroutine_{i}"] = {"inputs": inputs,
                                          "outputs": outputs,
                                          "program": program}
    return {"original_task_key": key,
            "subroutines": subroutines}

In [14]:
# we need arc original data, solvers.py, incremental_arc_dataset.json, dsl.py. ========================================

data = get_data()
import json
map = map_function_returns("rewritten_solvers.py")

with open("incremental_arc_dataset.json", 'r') as fp:
    incremental = json.load(fp)

with open("dsl.py", "r") as f:
    module_content = f.read()
evaluator = StepEvaluator()

#map_to_solutions = single_generate_solutions(map)

In [ ]:
def get_mutation_data(mutation_file):
    '''
    this is the function to get the mutation data. The format should be same as get_data()
    '''
    # Load mutation solutions
    with open(mutation_file, 'r') as f:
        data = json.load(f)

    # Convert function matching original get_data() format
    ast = lambda g: tuple(tuple(r) for r in g)
    
    processed_data = {}
    for task_id, task_data in data.items():
        # Format like original ARC JSON structure
        processed_data[task_id] = {
            'train': task_data['training_examples'],
            'test': task_data['test_examples']
        }

    # Now format exactly like get_data() return structure
    return {
        'train': {k: [{
            'input': ast(e['input']),
            'output': ast(e['output']),
        } for e in v['train']] for k, v in processed_data.items()},
        'test': {k: [{
            'input': ast(e['input']),
            'output': ast(e['output']),
        } for e in v['test']] for k, v in processed_data.items()}
    }

# Usage:
new_data = get_mutation_data("mutated_tasks_train_9600.json")

# Now you can use this data exactly as if it came from get_data()
# It will have the same structure and tuple format

In [ ]:
def get_solver(mutation_file):
    '''
    this is the function to get solvers from mutation data. The format should be the same as solvers.py. Need transformation to rewritte_solvers.py
    '''
    # Load mutation solutions
    with open(mutation_file, 'r') as f:
        data = json.load(f)
        
    # Extract all solver functions
    solvers = {}
    for task_id, task_data in data.items():
        # Get the program definition directly 
        program = task_data['program']
        # Add to solvers dictionary
        solvers[task_id] = program

    return solvers

# Usage:
solvers = get_solver("mutated_tasks_train_9600.json")
with open("new_solvers.py", 'w') as f:
    for solver in solvers.values():
        f.write(solver + "\n\n")

In [22]:
map = map_function_returns("rewritten_solvers.py")

with open("dsl.py", "r") as f:
    module_content = f.read()
evaluator = StepEvaluator()

map_to_solutions = single_generate_solutions(map)

Generating solutions: 100%|██████████| 400/400 [07:05<00:00,  1.06s/it]  


In [23]:
all_solutions = []
for name in tqdm(map_to_solutions):
    if name in ['solve_a64e4611', 'solve_2dd70a9a']:
        continue
    solutions = map_to_solutions[name]
    for solution in solutions:
        processed = process_solution(solution, name)
        if processed is not None:
            all_solutions.append(processed)

100%|██████████| 400/400 [01:29<00:00,  4.45it/s] 


In [24]:
all_solutions[0]

{'original_task_key': '67a3c6ac',
 'subroutines': {'subroutine_0': {'inputs': [[((6, 6, 6, 2),
      (6, 1, 6, 2),
      (7, 2, 7, 2),
      (1, 7, 2, 2))],
    [((7, 7, 7, 6, 6, 6, 2),
      (6, 7, 1, 1, 7, 7, 1),
      (7, 7, 2, 1, 2, 6, 6),
      (2, 2, 7, 7, 7, 2, 2),
      (7, 2, 7, 1, 2, 7, 2),
      (6, 6, 6, 2, 2, 1, 1),
      (6, 2, 6, 6, 6, 6, 6))],
    [((1, 2, 7, 1, 1, 1),
      (2, 1, 7, 7, 2, 6),
      (2, 1, 2, 6, 2, 1),
      (1, 2, 1, 7, 6, 2),
      (2, 7, 1, 2, 7, 1),
      (2, 1, 6, 2, 7, 7))]],
   'outputs': [((2, 6, 6, 6), (2, 6, 1, 6), (2, 7, 2, 7), (2, 2, 7, 1)),
    ((2, 6, 6, 6, 7, 7, 7),
     (1, 7, 7, 1, 1, 7, 6),
     (6, 6, 2, 1, 2, 7, 7),
     (2, 2, 7, 7, 7, 2, 2),
     (2, 7, 2, 1, 7, 2, 7),
     (1, 1, 2, 2, 6, 6, 6),
     (6, 6, 6, 6, 6, 2, 6)),
    ((1, 1, 1, 7, 2, 1),
     (6, 2, 7, 7, 1, 2),
     (1, 2, 6, 2, 1, 2),
     (2, 6, 7, 1, 2, 1),
     (1, 7, 2, 1, 7, 2),
     (7, 7, 2, 6, 1, 2))],
   'program': 'temp_0 = vmirror(I)'}}}

In [27]:
all_solutions[1000]

{'original_task_key': '694f12f3',
 'subroutines': {'subroutine_0': {'inputs': [[((0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
      (0, 4, 4, 4, 4, 0, 0, 0, 0, 0),
      (0, 4, 4, 4, 4, 0, 0, 0, 0, 0),
      (0, 4, 4, 4, 4, 0, 0, 0, 0, 0),
      (0, 4, 4, 4, 4, 0, 0, 0, 0, 0),
      (0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
      (0, 0, 0, 4, 4, 4, 4, 4, 4, 0),
      (0, 0, 0, 4, 4, 4, 4, 4, 4, 0),
      (0, 0, 0, 4, 4, 4, 4, 4, 4, 0),
      (0, 0, 0, 4, 4, 4, 4, 4, 4, 0))],
    [((0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
      (0, 4, 4, 4, 4, 4, 0, 0, 0, 0),
      (0, 4, 4, 4, 4, 4, 0, 0, 0, 0),
      (0, 4, 4, 4, 4, 4, 0, 0, 0, 0),
      (0, 4, 4, 4, 4, 4, 0, 0, 0, 0),
      (0, 4, 4, 4, 4, 4, 0, 0, 0, 0),
      (0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
      (0, 0, 0, 0, 0, 4, 4, 4, 4, 0),
      (0, 0, 0, 0, 0, 4, 4, 4, 4, 0),
      (0, 0, 0, 0, 0, 4, 4, 4, 4, 0))]],
   'outputs': [[[[4, [3, 1]],
      [4, [4, 2]],
      [4, [3, 3]],
      [4, [2, 1]],
      [4, [3, 2]],
      [4, [4, 1]],
      [4, [3, 4]],
      [4, [1, 1]

In [26]:
len(all_solutions)

13062

In [ ]:
tests = ['solve_9d9215db', 'solve_150deff5', 'solve_b7249182']
test_keys = [t.replace("solve_", "") for t in tests]
test_examples = [soln for soln in all_solutions if soln['original_task_key'] in test_keys]
train_examples = [soln for soln in all_solutions if soln['original_task_key'] not in test_keys]
decomposed = {"train": train_examples, 'test': test_examples}

In [16]:
def find_non_serializable(obj, path=""):
    """
    Recursively traverse an object and find all paths that contain non-JSON-serializable items.
    Specifically looks for function objects that would cause TypeError.
    
    Args:
        obj: The object to inspect
        path: Current path in the object (used recursively)
        
    Returns:
        list: List of paths (strings) where functions were found
    """
    problems = []
    
    # Handle different types
    if isinstance(obj, dict):
        for key, value in obj.items():
            new_path = f"{path}.{key}" if path else str(key)
            if callable(value):
                problems.append((new_path, type(value).__name__))
            else:
                problems.extend(find_non_serializable(value, new_path))
                
    elif isinstance(obj, (list, tuple, set)):
        for i, item in enumerate(obj):
            new_path = f"{path}[{i}]"
            if callable(item):
                problems.append((new_path, type(item).__name__))
            else:
                problems.extend(find_non_serializable(item, new_path))
                
    elif callable(obj):
        problems.append((path, type(obj).__name__))
        
    return problems

def debug_json_serialization(obj):
    """
    Wrapper function that provides a friendly output of all non-serializable paths found.
    
    Args:
        obj: The object to inspect
        
    Returns:
        None: Prints the problems found
    """
    problems = find_non_serializable(obj)
    
    if not problems:
        print("No serialization problems found!")
        return
        
    print("Found the following non-serializable items:")
    for path, type_name in problems:
        print(f"• At path '{path}': Found {type_name}")
    return problems

In [ ]:
probs = debug_json_serialization(decomposed)

In [ ]:
def set_default(obj):
    if isinstance(obj, (set, frozenset)):
        return list(obj)
    raise TypeError

In [ ]:
# Save the decomposed solutions to a JSON file
with open("arc_decomposed.json", 'w') as f:
    json.dump(decomposed, f, default=set_default)

In [ ]:
with open("arc_decomposed.json", 'r') as fp:
    solns = json.load(fp)

In [ ]:
def convert_to_tuples(list_of_lists):
    return tuple(tuple(t) for t in list_of_lists)


In [ ]:
convert_to_tuples(solns['train'][1000]['subroutines']['subroutine_0']['outputs'][0][0])

In [ ]:
charset = set()
step = 0
for example in solns['train']:
    for subroutine_name in example['subroutines']:
        charset.update(set(str(example['subroutines'][subroutine_name]['outputs'])))
        if "-" in str(example['subroutines'][subroutine_name]['outputs']):
            print(str(example['subroutines'][subroutine_name]['outputs']))

### Middle Plot

In [18]:
all_solutions = []
for name in tqdm(map_to_solutions):
    if name in ['solve_a64e4611', 'solve_2dd70a9a']:
        continue
    solutions = map_to_solutions[name]
    for solution in solutions:
        processed = process_solution(solution, name)
        if processed is not None:
            all_solutions.append(processed)

  0%|          | 0/8427 [00:00<?, ?it/s]


KeyError: 'f8b3ba0a_it0_0'